# Assignment 3: Answers

In this assignment, you will build a neural dependency parser using PyTorch. In Part 1, you will learn
about two general neural network techniques (Adam Optimization and Dropout) that you will use to build
the dependency parser in Part 2. In Part 2, you will implement and train the dependency parser, before
analyzing a few erroneous dependency parses.

## 1. Machine Learning & Neural Networks (8 points)

(a) (4 points) Adam Optimizer
Recall the standard Stochastic Gradient Descent update rule:

$$ \theta \leftarrow \theta - \alpha \nabla_\theta J_{minibatch}(\theta)$$

where $\theta$ is a vector containing all of the model parameters, $J$ is the loss function, $\nabla \theta J_{minibatch}(\theta)$ is the gradient of the loss function with respect to the parameters on a minibatch of data, and $\alpha$ is the learning rate. 

[Adam Optimization](https://arxiv.org/pdf/1412.6980.pdf) uses a more sophisticated update rule with two additional steps:

**i.(2 points)** First, *Adam* uses a trick called momentum by keeping track of m, a rolling average
of the gradients:
\begin{align}
m &\leftarrow \beta_1 m + (1-\beta_1)  \nabla_\theta J_{minibatch}(\theta)\\
\theta &\leftarrow \theta - \alpha m
\end{align}
where $β_1$ is a hyperparameter between 0 and 1 (often set to 0.9). Briefly explain (you don’t need
to prove mathematically, just give an intuition) how using m stops the updates from varying
as much and why this low variance may be helpful to learning, overall.

<font color='red'>Answer</font>: The momentum tracks a "history" of the last movement in the hyperparameter space. Then, by using a weighted average by* $\beta_1$, we just use an "amount" of the new change produced by the gradient, and not totally changing the overall direction in one step.

This technique avoids "zigzags" in the optimization process.

**ii. (2 points)** Adam also uses adaptive learning rates by keeping track of v, a rolling average of
the magnitudes of the gradients:
\begin{align}
m &\leftarrow \beta_1 m + (1-\beta_1)  \nabla_\theta J_{minibatch}(\theta)\\
v &\leftarrow \beta_2 v + (1-\beta_2)  \nabla_\theta J_{minibatch}(\theta) \odot \nabla_\theta J_{minibatch}(\theta)\\
θ &← θ − α \odot \frac{m}{\sqrt{v}}\\
\end{align}

where $\odot$ and / denote elementwise multiplication and division (so $z \odot z$ is elementwise squaring)
and $\beta_2$ is a hyperparameter between 0 and 1 (often set to 0.99). Since Adam divides the update
by $\sqrt{v}$, which of the model parameters will get larger updates? Why might this help with
learning?


<font color='red'>Answer</font>: 
We can, in an abuse of notation, say that: 
$$
\nabla_\theta J_{minibatch}(\theta) \odot \nabla_\theta J_{minibatch}(\theta) = \nabla_\theta J_{minibatch}(\theta)^2
$$

That is, each component of the gradient is squared. 

So, the recurrent equation of $v$ is just a moving average of the squares of each component. We can see it as a second moment estimation of the gradient, that is, if we think that the gradient is a random vector $X$ we are computing the uncentered variance of each component ($E[X^2]$). Recall that the variance of a random variable is $Var[X] = E[X^2] - E[X]^2$.

Dividing by the square root of the variance (that is, the deviation) will normalize the values of $v$. Thus, a value with a tiny derivative will have the same step as one with a big derivative, smoothing the step across the parameters. (Am I sure of this?))

In proportion, values with a small variance will likely receive a bigger update, helping to get them off of possibly saddle points or local minima. Also, values will large variances will receive proportionally smaller updates to prevent overshooting in a wrong direction.


**b)(4 points)** Dropout 3 is a regularization technique. During training, dropout randomly sets units
in the hidden layer $h$ to zero with probability $p_{drop}$ (dropping different units each minibatch), and
then multiplies h by a constant $\gamma$. We can write this as

$$
h_{drop} = \gamma d \odot h
$$

**i. (2 points)** What must γ equal in terms of p drop ? Briefly justify your answer.

<font color='red'>Answer</font>: 
We want $E[h_{drop}]_i = h_i$, 

$$
h_{drop, i} =  \gamma d_i h_i$$

Then,
$$
E[h_{drop, i}] =  \gamma (1 - p_{drop}) h_i
$$

To make this product equal to $h_i$, we need $$\gamma = \frac{1}{1-p_{drop}}$$



**ii. (2 points)** Why should we apply dropout during training but not during evaluation?

Because we want to use all the neurons we have trained

## 2. Neural Transition-based dependency parsing

We’d like to look at example dependency parses and understand where parsers like ours
might be wrong. For example, in this sentence:

<img src="imgs/moscow_error.png"/>

the dependency of the phrase *into Afghanistan* is wrong, because the phrase should modify sent (as
in sent into Afghanistan) not troops (because troops into Afghanistan doesn’t make sense). Here is
the correct parse:

<img src="imgs/moscow_ok.png"/>


There are four types of parsing error:
- **Prepositional Phrase Attachment Error**: In the example above, the phrase into Afghanistan
is a prepositional phrase. A Prepositional Phrase Attachment Error is when a prepositional
phrase is attached to the wrong head word (in this example, troops is the wrong head word and
sent is the correct head word). More examples of prepositional phrases include with a rock,
before midnight and under the carpet.
- **Verb Phrase Attachment Error**: In the sentence *Leaving the store unattended, I went
outside to watch the parade*, the phrase *leaving the store* unattended is a verb phrase. A Verb
Phrase Attachment Error is when a verb phrase is attached to the wrong head word (in this
example, the correct head word is went).
- **Modifier Attachment Error**: In the sentence I am extremely short, the adverb extremely is
a modifier of the adjective short. A Modifier Attachment Error is when a modifier is attached
to the wrong head word (in this example, the correct head word is short).
- **Coordination Attachment Error**: In the sentence Would you like brown rice or garlic naan?,
the phrases brown rice and garlic naan are both conjuncts and the word or is the coordinating
conjunction. The second conjunct (here garlic naan) should be attached to the first conjunct
(here brown rice). A Coordination Attachment Error is when the second conjunct is attached
to the wrong head word (in this example, the correct head word is rice). Other coordinating
conjunctions include and, but and so.


In this question are four sentences with dependency parses obtained from a parser. Each sentence
has one error, and there is one example of each of the four types above. For each sentence, state
the type of error, the incorrect dependency, and the correct dependency. To demonstrate: for the
example above, you would write:

<img src="imgs/error_1.png"/>



<font color='red'>Answer</font>: There is a verb phrase (*fearing my death*) incorrectly attached to *wedding*. So the error here is one of the **Verb Phrase Attachment Error**

To correct it, we should remove the mentioned dependency. **fearing** should be dependent of **heading**

<img src="imgs/error_2.png"/>


<font color='red'>Answer</font>: Uhmmm.... I suppose this is a coordination error, but I haven't seen coordination errors so far.

<img src="imgs/error_3.png"/>


<font color='red'>Answer</font>: 

Error: **Prepositional Phrase Attachment Error**. "In Midland, Texas" is a prepositional phrase attached to the subject of the sentence ("It is on loan in Midland, Texas"). 

So, remove named--> Midland, and make Midland dependent of *loan*

<img src="imgs/error_4.png"/>


<font color='red'>Answer</font>: 

Error: ???